In [2]:
import pickle
import numpy as np
from scipy.io import wavfile
from util import audio
from hparams import hparams
from scipy import signal
import librosa
import librosa.display
import matplotlib.pyplot as plt


import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv1D, MaxPooling1D, Input, Layer
from tensorflow.keras.layers import Embedding
from tensorflow.keras import regularizers
from tqdm import tqdm

from absl import app
from absl import flags
from absl import logging

import os

small_data = 'data/lj/small.pickle'

#Load small test set
train_x = None
train_y = None
with open(small_data, "rb") as handle:
    train_x, train_y = pickle.load(handle)

C:\Users\jz124\Anaconda2\envs\py36\lib\site-packages\tensorflow\python\framework\dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\jz124\Anaconda2\envs\py36\lib\site-packages\tensorflow\python\framework\dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\jz124\Anaconda2\envs\py36\lib\site-packages\tensorflow\python\framework\dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\jz124\Anaconda2\envs\py36\lib\site-packages\tensorflow\python\frame

In [13]:
#Training Parameters
num_epochs = 100
steps_per_epoch = None
batch_size = 32
loss = None
optimizer = None
lr = 0.001
filters = 512
sample_rate = 22050 #Hz 16 bit PCM WAV files
wav_dir = './LJSPEECH-1.1/'
out_dir = './output'
os.makedirs(out_dir, exist_ok=True)
hop_length = int(hparams.frame_shift_ms / 1000 * hparams.sample_rate) #change
n_mels = 128
n_fft = 2048


In [14]:
print(train_y[0])
wavfile.write("test.wav", sample_rate, train_y[0])

[-24 -25 -21 ...  24   7   2]


In [26]:
#####PREPROCESSING#######

ind = 1
with open(os.path.join(wav_dir, 'metadata.csv'), encoding='utf-8') as f:
    for inp in f:
        data = inp.strip().split('|')
        wav_file = os.path.join(wav_dir, 'wavs/' + data[0] + '.wav')
        text = data[2]
        rate, wav = wavfile.read(wav_file)
        wav = wav.astype('float32')
        
        # normalize
        wav /= wav.max()
        
        #Compute Spectrogram
        fft = np.abs(librosa.stft(wav, n_fft=n_fft, hop_length=hop_length))
        spectrogram = librosa.amplitude_to_db(fft, ref=np.max)
        
        n_frames = spectrogram[1]
        

        #Compute Melspectrogram
        mel_spectrogram = librosa.feature.melspectrogram(y = wav, sr = sample_rate, n_fft=n_fft, 
                                                         hop_length=hop_length, n_mels=n_mels)
        # melspectrogram on log scale
        mel_spectrogram = librosa.power_to_db(S, ref=np.max)
        spectrogram_filename = 'ljspeech-spec-%05d.npy' % ind
        mel_filename = 'ljspeech-mel-%05d.npy' % ind
        np.save(os.path.join(out_dir, spectrogram_filename), spectrogram.T, allow_pickle=False)
        np.save(os.path.join(out_dir, mel_filename), mel_spectrogram.T, allow_pickle=False)
        ind += 1
        
